# The Greenhouse Gas (GHS) emissions

In [1]:
# HIDDEN
%load_ext autoreload
%autoreload 2

## Get GHG emission from agricultural land-use

In [2]:
# HIDDEN
import altair as alt

from tools import get_all_files, get_GHG_file_df
from tools.Plot_GHG import get_GHG_plots,plot_GHG_total

/Users/gigi/miniconda3/envs/jbook/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data preparing

In [3]:
# HIDE CODE
# Get all output files
from PARAMETERS import DATA_ROOT
files = get_all_files(DATA_ROOT)


# Select files based on criteria
files_selc = files.query('year <= 2050')
print(files_selc['catetory'].unique(),'\n')

['GHG' 'dvar' 'ammap' 'lmmap' 'lumap_separate_Agricultural Land-use'
 'lumap_separate_Non-Agricultural Land-use' 'lumap' 'water' 'cross_table'
 'quantity'] 



Filter the GHG data

In [4]:
# NO CODE
# Get the GHG files
GHG_files = get_GHG_file_df(files_selc)
GHG_files = GHG_files.reset_index(drop=True).sort_values(['year','GHG_sum_t'])
GHG_files['GHG_sum_Mt'] = GHG_files['GHG_sum_t'] / 1e6
GHG_files

,year,catetory,base_name,base_ext,path,GHG_sum_t,GHG_sum_Mt
2,2010,GHG,Non-Agricultural Landuse,.csv,../LUTO_DATA/2023_11_21__05_26_04__hard_mincos...,-1.077782e+05,-0.107778
1,2010,GHG,Agricultural Management,.csv,../LUTO_DATA/2023_11_21__05_26_04__hard_mincos...,0.000000e+00,0.000000
3,2010,GHG,Transition Penalty,.csv,../LUTO_DATA/2023_11_21__05_26_04__hard_mincos...,0.000000e+00,0.000000
0,2010,GHG,Agricultural Landuse,.csv,../LUTO_DATA/2023_11_21__05_26_04__hard_mincos...,9.051190e+07,90.511903
6,2011,GHG,Non-Agricultural Landuse,.csv,../LUTO_DATA/2023_11_21__05_26_04__hard_mincos...,-8.273422e+06,-8.273422
...,...,...,...,...,...,...,...
156,2049,GHG,Agricultural Landuse,.csv,../LUTO_DATA/2023_11_21__05_26_04__hard_mincos...,7.590731e+07,75.907306
162,2050,GHG,Non-Agricultural Landuse,.csv,../LUTO_DATA/2023_11_21__05_26_04__hard_mincos...,-3.799907e+08,-379.990720
161,2050,GHG,Agricultural Management,.csv,../LUTO_DATA/2023_11_21__05_26_04__hard_mincos...,-3.439199e+07,-34.391990
163,2050,GHG,Transition Penalty,.csv,../LUTO_DATA/2023_11_21__05_26_04__hard_mincos...,6.538516e+04,0.065385


Calculate the net emissions

In [5]:
# HIDDEN
# Calculate the net emissions
Net_emission = GHG_files.groupby('year')['GHG_sum_Mt'].sum(numeric_only = True).reset_index()
Net_emission = Net_emission.rename(columns={'GHG_sum_Mt':'Net_emission'})

Net_emission['Net_emission_cum'] = Net_emission['Net_emission'].cumsum()
Net_emission.head(3)

,year,Net_emission,Net_emission_cum
0,2010,90.404125,90.404125
1,2011,79.323406,169.727530
2,2012,68.648003,238.375533


## Total GHG emissions across years

In [6]:
# HIDE CODE  
plot_GHG_total(GHG_files, Net_emission)

alt.LayerChart(...)

## Agricultural Land use Emissions

Get the Agricultural land-use data

In [7]:
# HIDDEN
# Initialize the class
ag_lucc_GHG = get_GHG_plots(GHG_files,'Agricultural Landuse','GHG emissions (Mt CO2e)')

### Emissions by Crops/Livestocks

In [8]:
# HIDE CODE
# get the df and plot
GHG_crop_lvstk_df,GHG_crop_lvstk_plot = ag_lucc_GHG.plot_GHG_crop_lvstk()
GHG_crop_lvstk_plot

alt.LayerChart(...)

### Emissions by differrnt irrigation system

In [9]:
# HIDE CODE
# get the df and plot
GHG_dry_irr_df,GHG_dry_irr_plot = ag_lucc_GHG.plot_GHG_dry_irr()
GHG_dry_irr_plot

alt.LayerChart(...)

### Emissions by GHG category

In [27]:
# HIDE CODE
# get the df and plot
GHG_catetory_df,GHG_atetory_plot = ag_lucc_GHG.plot_GHG_category()
GHG_atetory_plot

alt.LayerChart(...)

### Emission by CO2 sources

In [11]:
# HIDE CODE
# get the df and plot
GHG_sources_df,GHG_sources_plot = ag_lucc_GHG.plot_GHG_sources()
GHG_sources_plot

alt.LayerChart(...)

### Emissions by {land-use}-{irrigation} cmobined

In [12]:
# HIDE CODE
# get the df and plot
GHG_lu_lm_df,GHG_lu_lm_plot = ag_lucc_GHG.plot_GHG_lu_lm(2050)
GHG_lu_lm_plot

alt.Chart(...)

### Emissions by {land-use}-{GHG source} combined

In [13]:
# HIDE CODE
# get the df and plot
GHG_lu_source_df,GHG_lu_source_plot = ag_lucc_GHG.plot_GHG_lu_source(2050)
GHG_lu_source_plot

alt.Chart(...)

## Non-Agricultural Land-use GHG sequestrations

### Data preparation

In [14]:
# HIDDEN
# Initialize the class
non_ag_lucc_GHG = get_GHG_plots(GHG_files,'Non-Agricultural Landuse','GHG sequestration (Mt CO2e)')

/Users/gigi/Desktop/VIS_LUTO/tools/Plot_GHG.py:229: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([GHG_CO2,GHG_nonCO2],axis=0).reset_index(drop=True)


In [15]:
non_ag_lucc_GHG.GHG_df_long

,Year,Land use,Land category,Land use category,Irrigation,Sources,Quantity (Mt CO2e),GHG Category
0,2011,Environmental Plantings,Non-Agricultural Landuse,Non-Agricultural Landuse,dry,TCO2E_Environmental Planting,-8.113680,CO2
1,2012,Environmental Plantings,Non-Agricultural Landuse,Non-Agricultural Landuse,dry,TCO2E_Environmental Planting,-17.694736,CO2
2,2013,Environmental Plantings,Non-Agricultural Landuse,Non-Agricultural Landuse,dry,TCO2E_Environmental Planting,-27.362048,CO2
3,2014,Environmental Plantings,Non-Agricultural Landuse,Non-Agricultural Landuse,dry,TCO2E_Environmental Planting,-37.123370,CO2
4,2015,Environmental Plantings,Non-Agricultural Landuse,Non-Agricultural Landuse,dry,TCO2E_Environmental Planting,-46.952744,CO2
...,...,...,...,...,...,...,...,...
75,2046,Environmental Plantings,Non-Agricultural Landuse,Non-Agricultural Landuse,irr,TCO2E_Environmental Planting,-7.520842,CO2
76,2047,Environmental Plantings,Non-Agricultural Landuse,Non-Agricultural Landuse,irr,TCO2E_Environmental Planting,-7.736231,CO2
77,2048,Environmental Plantings,Non-Agricultural Landuse,Non-Agricultural Landuse,irr,TCO2E_Environmental Planting,-8.345396,CO2
78,2049,Environmental Plantings,Non-Agricultural Landuse,Non-Agricultural Landuse,irr,TCO2E_Environmental Planting,-9.078595,CO2


### Sequestrations in total

In [28]:
# HIDE CODE
# get the df and plot
GHG_non_ag_crop_lvstk_df,GHG_non_ag_crop_lvstk_plot = non_ag_lucc_GHG.plot_GHG_crop_lvstk()
GHG_non_ag_crop_lvstk_plot

alt.LayerChart(...)

### Sequestrations by irrigation system

In [17]:
# # HIDE CODE
# # get the df and plot
# GHG_non_ag_dry_irr_df,GHG_non_ag_dry_irr_plot = non_ag_lucc_GHG.plot_GHG_dry_irr()
# GHG_non_ag_dry_irr_plot

## Agricultural Management GHG sequestrations

### Data preparation

In [18]:
# HIDDEN
# Initialize the class
ag_man_GHG = get_GHG_plots(GHG_files,'Agricultural Management','GHG sequestration (Mt CO2e)')

In [19]:
ag_man_GHG.GHG_df_long

,Year,Land use,Land category,Land use category,Irrigation,Sources,Quantity (Mt CO2e),GHG Category
0,2010,Apples,Crop,Crop,dry,Asparagopsis taxiformis,0.0,Asparagopsis taxiformis
1,2010,Citrus,Crop,Crop,dry,Asparagopsis taxiformis,0.0,Asparagopsis taxiformis
2,2010,Cotton,Crop,Crop,dry,Asparagopsis taxiformis,0.0,Asparagopsis taxiformis
3,2010,Grapes,Crop,Crop,dry,Asparagopsis taxiformis,0.0,Asparagopsis taxiformis
4,2010,Hay,Crop,Crop,dry,Asparagopsis taxiformis,0.0,Asparagopsis taxiformis
...,...,...,...,...,...,...,...,...
6361,2050,Beef,natural land,Livestock,irr,Ecological Grazing,0.0,Ecological Grazing
6362,2050,Dairy,modified land,Livestock,irr,Ecological Grazing,0.0,Ecological Grazing
6363,2050,Dairy,natural land,Livestock,irr,Ecological Grazing,0.0,Ecological Grazing
6364,2050,Sheep,modified land,Livestock,irr,Ecological Grazing,0.0,Ecological Grazing


### Sequestrations by Crops/Livestocks

In [23]:
# HIDE CODE
# get the df and plot
GHG_ag_man_GHG_crop_lvstk_df,GHG_ag_man_GHG_crop_lvstk_plot = ag_man_GHG.plot_GHG_crop_lvstk()
GHG_ag_man_GHG_crop_lvstk_plot

alt.LayerChart(...)

### Sequestrations by irrigation system

In [24]:
# HIDE CODE
# get the df and plot
GHG_ag_man_dry_irr_df,GHG_ag_man_dry_irr_plot = ag_man_GHG.plot_GHG_dry_irr()
GHG_ag_man_dry_irr_plot

alt.LayerChart(...)

### Sequestrations by GHG category

In [26]:
# HIDE CODE
# get the df and plot
GHG_ag_man_df,GHG_ag_man_plot = ag_man_GHG.plot_GHG_category()
GHG_ag_man_plot

alt.LayerChart(...)